# Example usage of gw_eccentricity

### Get the available methods

We implement different methods to measure eccentricity and mean anomaly

In [1]:
from gw_eccentricity.gw_eccentricity import get_available_methods

ModuleNotFoundError: No module named 'gw_eccentricity'